In [1]:
import gym
import random
from collections import deque
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchmetrics.functional import mean_squared_error

In [47]:
import torch
t = torch.tensor([1e5, 2e4, 1e-1])
torch.nn.functional.normalize(t, dim=0, p=2)

tensor([9.8058e-01, 1.9612e-01, 9.8058e-07])

In [11]:
env_name = 'MountainCar-v0'
env = gym.make(env_name)
env.seed(0)
print(f'{env.observation_space=}\n{env.observation_space.shape=}\n{env.action_space=}\n{env.action_space.n=}')

env.observation_space=Box([-1.2  -0.07], [0.6  0.07], (2,), float32)
env.observation_space.shape=(2,)
env.action_space=Discrete(3)
env.action_space.n=3


In [3]:
class QNetwork(nn.Module):
    def __init__(self, state_dim, action_size, hidden_dim=100):
        super().__init__()
        self.hidden1 = nn.Linear(state_dim, hidden_dim)
        self.hidden2 = nn.Linear(hidden_dim, hidden_dim)
        self.q_state = nn.Linear(hidden_dim, action_size)
        
    def forward(self, state):
        x = F.elu(self.hidden1(state))
        x = F.elu(self.hidden2(x))
        x = self.q_state(x)
        return x

In [4]:
class ReplayBuffer():
    def __init__(self, maxlen):
        self.buffer = deque(maxlen=maxlen)
        self.last_experience = None
        
    def add(self, experience):
        self.buffer.append(experience)
        self.last_experience = experience
        
    def sample(self, batch_size, **kwargs):
        sample_size = min(len(self.buffer), batch_size)
        samples = random.choices(self.buffer, k=sample_size-1)
        samples.append(self.last_experience)
        zipped = zip(*samples)
        if torch.cuda.is_available():
            tensors = map(lambda x: torch.tensor(x).cuda(), zipped)
        else:
            tensors = map(torch.tensor, zipped)
        # all have importance of 1
        return tensors, 1

In [6]:
class PriorityReplayBuffer():
    def __init__(self, maxlen):
        self.buffer = np.zeros(maxlen, dtype=object)
        self.priorities = np.zeros(maxlen, dtype=float)
        self.importance = np.zeros(maxlen, dtype=float)
        self.sum_priority = 0
        self.max_importance = 0
        # helps us iterate the head of the Turing Machine tape (fixed length of 'maxlen')
        self.iterator = 0
        # iterator will track the 'true' length of the buffer (instead of explicitly using a deque)
        self.true_len = 0
        
    def add(self, experience):
        self.true_len += 1
        # the object's true size is the minimum between the true_len iterator and the internal buffer size (which starts at maxlen)
        self.true_len = min(self.true_len, len(self.buffer))
        iterator = self.iterator
        self.buffer[iterator] = experience
        prev_priority = self.priorities[iterator]
        new_priority = 1.
        self.priorities[iterator] = new_priority
        new_importance = 1. / self.true_len
        self.importance[iterator] = new_importance
        self.sum_priority += new_priority - prev_priority
        self.max_importance = max(self.max_importance, new_importance)
        self.iterator = iterator + 1 if iterator < len(self.buffer) - 1 else 0
        
    def sample(self, batch_size, **kwargs):
        len_buffer = self.true_len
        sample_size = min(len_buffer, batch_size)
        sample_probs = self.priorities[:len_buffer] / self.sum_priority
        # we remember these in an obj attribute to use them at the set_priotities stage (which needs the TD error to be computed first)
        self.sample_indices = np.random.default_rng().choice(range(len_buffer), size=sample_size, p=sample_probs)
        samples = self.buffer[self.sample_indices]
        importance = self.importance[self.sample_indices] / self.max_importance
        
        zipped = zip(*samples)
        if torch.cuda.is_available():
            tensors = map(lambda x: torch.tensor(x).cuda(), zipped)
            importance = torch.tensor(importance).cuda()
        else:
            tensors = map(torch.tensor, zipped)
            importance = torch.tensor(importance)
        return tensors, importance
    
    def set_priorities(self, errors, priority_scale=0.7, offset=0.1):
        for i,e in zip(self.sample_indices, errors):
            prev_priority = self.priorities[i]
            raw_new_priority = abs(e.item()) + offset
            new_priority = self.priorities[i] = raw_new_priority ** priority_scale
            self.sum_priority += new_priority - prev_priority
            new_importance = self.importance[i] = 1 / (self.true_len * raw_new_priority)
            self.max_importance = max(self.max_importance, new_importance)

In [31]:
class PriorityReplayBufferInef():
    def __init__(self, maxlen):
        self.buffer = deque(maxlen=maxlen)
        self.priorities = deque(maxlen=maxlen)
        
    def add(self, experience):
        self.buffer.append(experience)
        self.priorities.append(max(self.priorities, default=1))
        
    def get_probabilities(self, priority_scale=0.7):
        scaled_priorities = np.array(self.priorities) ** priority_scale
        sample_probabilities = scaled_priorities / sum(scaled_priorities)
        return sample_probabilities
    
    def get_importance(self, probabilities):
        importance = 1/len(self.buffer) * 1/probabilities
        importance_normalized = importance / max(importance)
        return importance_normalized
        
    def sample(self, batch_size, priority_scale=0.7):
        sample_size = min(len(self.buffer), batch_size)
        sample_probs = self.get_probabilities(priority_scale)
        # we remember these in an obj attribute to use them at the set_priotities stage (which needs the TD error to be computed first)
        self.sample_indices = random.choices(range(len(self.buffer)), k=sample_size, weights=sample_probs)
        samples = np.array(self.buffer, dtype=object)[self.sample_indices]
        importance = self.get_importance(sample_probs[self.sample_indices])
        
        zipped = zip(*samples)
        if torch.cuda.is_available():
            tensors = map(lambda x: torch.tensor(x).cuda(), zipped)
            importance = torch.tensor(importance).cuda()
        else:
            tensors = map(torch.tensor, zipped)
            importance = torch.tensor(importance)
        return tensors, importance
    
    def set_priorities(self, errors, offset=0.1):
        for i,e in zip(self.sample_indices, errors):
            self.priorities[i] = abs(e.item()) + offset

In [29]:
class DQNAgent():
    def __init__(self, env, hidden_dim=100, gamma=.99, eps_decay=.99, lr=.001, priority_buffer=False, len_buffer=1000, batch_size=100, use_different_target=False):
        self.state_dim = env.observation_space.shape[0]
        self.action_size = env.action_space.n
        self.q_network = QNetwork(self.state_dim, self.action_size, hidden_dim)
        self.q_network_target = QNetwork(self.state_dim, self.action_size, hidden_dim) if use_different_target else self.q_network
        if torch.cuda.is_available():
            self.q_network = self.q_network.cuda()
            self.q_network_target = self.q_network_target.cuda()
        self.priority_buffer = priority_buffer
        self.replay_buffer = PriorityReplayBufferInef(maxlen=len_buffer) if priority_buffer else ReplayBuffer(maxlen=len_buffer)
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=lr)
        self.gamma = gamma
        self.eps = 1.
        self.eps_decay = eps_decay
        self.batch_size = batch_size
        
    def get_action(self, state):
        state = torch.tensor(state)
        if torch.cuda.is_available():
            state = state.cuda()
        q_state = self.q_network(state)
        action = random.randint(0, self.action_size - 1) if random.random() < self.eps else torch.argmax(q_state).item()
        return action
    
    def train(self, state, action, next_state, reward, done):
        # add to replay buffer, and sample a batch of transitions
        self.replay_buffer.add((state, action, next_state, reward, done))
        (states, actions, next_states, rewards, dones), importance = self.replay_buffer.sample(self.batch_size)
        if torch.cuda.is_available():
            states = states.cuda()
            next_states = next_states.cuda()
        # get Q values for all actions possible in current state
        q_states = self.q_network(states)
        # select Q value for the specific action that was taken (vectorized)
        q_state_actions = q_states.gather(1, actions.view(-1, 1)).squeeze(dim=-1)
        # get Q target from next_states (according to DQN this should be lagging behind, with no grad updates)
        with torch.no_grad():
            q_next_states = self.q_network_target(next_states)
            q_next_states[dones] = 0
            q_targets = rewards + self.gamma * torch.max(q_next_states, dim=1).values
        self.optimizer.zero_grad()
        errors = q_targets - q_state_actions
        # the importance needs to be scaled by 'b' as per the paper
        # 'b' starts at 0, which reflects normal updates at the beginning where mostly random actions are picked
        # and then increases over time to 1, to account for the bias introduced due to priority sampling the experiences
        importance_factor = importance ** (1 - self.eps) if self.priority_buffer else 1
        loss = torch.mean(importance_factor * errors ** 2)
        loss.backward()
        self.optimizer.step()
        
        if self.priority_buffer:
            # set the new priorities in the replay buffer based on the td errors
            self.replay_buffer.set_priorities(errors)
        if done:
            # decay eps over time
            self.eps = max(.1, self.eps_decay * self.eps)

In [ ]:
from itertools import count
import time

priority_buffer = True
use_different_target = True
update_target_after = 30
hidden_dim = 32
eps_decay = .995
gamma = .95
maxlen = 1000
agent = DQNAgent(env, hidden_dim=hidden_dim, gamma=gamma, priority_buffer=priority_buffer, len_buffer=maxlen, 
                 eps_decay=eps_decay, use_different_target=use_different_target)
num_episodes = 800
num_steps = 200
events_range = range(num_steps) if num_steps else count()
render = False
print_reward_every = 10

t1 = 0
ep_rewards = 0
try:
    for ep in range(num_episodes):
        state = env.reset()
        total_reward = 0
        done = False
        
        if use_different_target and not ep % update_target_after:
            agent.q_network_target.load_state_dict(agent.q_network.state_dict())

        for t in events_range:
            action = agent.get_action(state)
            next_state, reward, done, info = env.step(action)
            agent.train(state, action, next_state, reward, done)
            total_reward += reward
            state = next_state
            if render:
                env.render()
            if done:
                break

        ep_rewards += total_reward
        if not ep % print_reward_every:
            t2 = time.time()
            print(f'Episode {ep}, total reward: {total_reward}, average since last: {ep_rewards / print_reward_every}; computed in: {t2 - t1}')
            t1 = t
            ep_rewards = 0
            
finally:
    env.close()

In [ ]:
agent.eps = 0
agent.q_network.eval()
agent.q_network_target.eval()
render = True
try:
    for ep in range(num_episodes):
        state = env.reset()
        total_reward = 0
        done = False

        for time in events_range:
            action = agent.get_action(state)
            next_state, reward, done, info = env.step(action)
            total_reward += reward
            state = next_state
            if render:
                env.render()
            if done:
                break
        
        print(f'Episode {ep}, in {time} steps; total reward: {total_reward}')
finally:
    env.close()

<h1> Frozen lake 

In [1]:
import numpy as np
import random
import time
import gym
from gym import wrappers

def run_episode(env, policy, episode_len=100):
    total_reward = 0
    obs = env.reset()
    for t in range(episode_len):
        # env.render()
        action = policy[obs]
        obs, reward, done, _ = env.step(action)
        total_reward += reward
        if done:
            # print('Epside finished after {} timesteps.'.format(t+1))
            break
    return total_reward


def run_policy(env, policy, num_ep=200, monitor=False):
    if monitor:
        env = wrappers.Monitor(env, 'saved/frozenlake1', video_callable=lambda episode_id: True, force=True)
    for _ in range(num_ep):
        run_episode(env, policy)
    env.close()    


def evaluate_policy(env, policy, n_episodes=100):
    total_rewards = 0.0
    for _ in range(n_episodes):
        total_rewards += run_episode(env, policy)
    return total_rewards / n_episodes

def gen_random_policy():
    return np.random.choice(4, size=((16)))

def crossover(policy1, policy2):
    new_policy = policy1.copy()
    for i in range(16):
        rand = np.random.uniform()
        if rand > 0.5:
            new_policy[i] = policy2[i]
    return new_policy

def mutation(policy, p=0.05):
    new_policy = policy.copy()
    for i in range(16):
        rand = np.random.uniform()
        if rand < p:
            new_policy[i] = np.random.choice(4)
    return new_policy

def main(n_steps=20, n_eval_steps=200):
    random.seed(1234)
    np.random.seed(1234)
    env = gym.make('FrozenLake-v1')
    env.seed(0)
    # env = wrappers.Monitor(env, '/tmp/frozenlake1', force=True)
    ## Policy search
    n_policy = 100
    start = time.time()
    policy_pop = [gen_random_policy() for _ in range(n_policy)]
    for idx in range(n_steps):
        policy_scores = [evaluate_policy(env, p) for p in policy_pop]
        print('Generation %d : max score = %0.2f' %(idx+1, max(policy_scores)))
        policy_ranks = list(reversed(np.argsort(policy_scores)))
        elite_set = [policy_pop[x] for x in policy_ranks[:5]]
        select_probs = np.array(policy_scores) / np.sum(policy_scores)
        child_set = [crossover(
            policy_pop[np.random.choice(range(n_policy), p=select_probs)], 
            policy_pop[np.random.choice(range(n_policy), p=select_probs)])
            for _ in range(n_policy - 5)]
        mutated_list = [mutation(p) for p in child_set]
        policy_pop = elite_set
        policy_pop += mutated_list
    policy_score = [evaluate_policy(env, p) for p in policy_pop]
    best_policy = policy_pop[np.argmax(policy_score)]

    end = time.time()
    print('Best policy score = %0.2f. Time taken = %4.4f'
            %(np.max(policy_score), (end-start)))    

    ## Evaluation
    run_policy(env, best_policy, n_eval_steps, monitor=True)
    return best_policy

In [2]:
policy = main(n_steps=5, n_eval_steps=50)

Generation 1 : max score = 0.20
Generation 2 : max score = 0.30
Generation 3 : max score = 0.60
Generation 4 : max score = 0.66
Generation 5 : max score = 0.79
Best policy score = 0.84. Time taken = 13.1888


In [3]:
"""
Solving FrozenLake8x8 environment using Policy iteration.
Author : Moustafa Alzantot (malzantot@ucla.edu)
"""
import numpy as np
import gym
from gym import wrappers


def run_episode(env, policy, gamma = 1.0, render = False):
    """ Runs an episode and return the total reward """
    obs = env.reset()
    total_reward = 0
    step_idx = 0
    while True:
        if render:
            env.render()
        obs, reward, done , _ = env.step(int(policy[obs]))
        total_reward += (gamma ** step_idx * reward)
        step_idx += 1
        if done:
            break
    return total_reward


def evaluate_policy(env, policy, gamma = 1.0, n = 100):
    scores = [run_episode(env, policy, gamma, False) for _ in range(n)]
    return np.mean(scores)

def extract_policy(env, v, gamma = 1.0):
    """ Extract the policy given a value-function """
    policy = np.zeros(env.nS)
    for s in range(env.nS):
        q_sa = np.zeros(env.nA)
        for a in range(env.nA):
            q_sa[a] = sum([p * (r + gamma * v[s_]) for p, s_, r, _ in  env.P[s][a]])
        policy[s] = np.argmax(q_sa)
    return policy

def compute_policy_v(env, policy, gamma=1.0):
    """ Iteratively evaluate the value-function under policy.
    Alternatively, we could formulate a set of linear equations in iterms of v[s] 
    and solve them to find the value function.
    """
    v = np.zeros(env.nS)
    eps = 1e-10
    while True:
        prev_v = np.copy(v)
        for s in range(env.nS):
            policy_a = policy[s]
            v[s] = sum([p * (r + gamma * prev_v[s_]) for p, s_, r, _ in env.P[s][policy_a]])
        if (np.sum((np.fabs(prev_v - v))) <= eps):
            # value converged
            break
    return v

def policy_iteration(env, gamma = 1.0):
    """ Policy-Iteration algorithm """
    policy = np.random.choice(env.nA, size=(env.nS))  # initialize a random policy
    max_iterations = 200000
    gamma = 1.0
    for i in range(max_iterations):
        old_policy_v = compute_policy_v(env, policy, gamma)
        new_policy = extract_policy(env, old_policy_v, gamma)
        if (np.all(policy == new_policy)):
            print ('Policy-Iteration converged at step %d.' %(i+1))
            break
        policy = new_policy
    return policy


def main_pi():
    env_name  = 'FrozenLake8x8-v1'
    env = gym.make(env_name)
    optimal_policy = policy_iteration(env, gamma = 1.0)
    scores = evaluate_policy(env, optimal_policy, gamma = 1.0)
    print('Average scores = ', np.mean(scores))

In [4]:
main_pi()

Policy-Iteration converged at step 12.
Average scores =  0.82


In [10]:

"""
Q-Learning example using OpenAI gym MountainCar enviornment
Author: Moustafa Alzantot (malzantot@ucla.edu)
"""
import numpy as np

import gym
from gym import wrappers

n_states = 40
iter_max = 2000

initial_lr = 1.0 # Learning rate
min_lr = 0.003
gamma = 1.0
t_max = 200
eps = 0.02

def run_episode(env, policy=None, render=False):
    obs = env.reset()
    total_reward = 0
    step_idx = 0
    try:
        for _ in range(t_max):
            if render:
                env.render()
            if policy is None:
                action = env.action_space.sample()
            else:
                a,b = obs_to_state(env, obs)
                action = policy[a][b]
            obs, reward, done, _ = env.step(action)
            total_reward += gamma ** step_idx * reward
            step_idx += 1
            if done:
                break
    except:
        env.close()
    return total_reward

def obs_to_state(env, obs):
    """ Maps an observation to state """
    env_low = env.observation_space.low
    env_high = env.observation_space.high
    env_dx = (env_high - env_low) / n_states
    a = int((obs[0] - env_low[0])/env_dx[0])
    b = int((obs[1] - env_low[1])/env_dx[1])
    return a, b

def main_qlearn():
    env_name = 'MountainCar-v0'
    env = gym.make(env_name)
    env.seed(0)
    np.random.seed(0)
    print ('----- using Q Learning -----')
    q_table = np.zeros((n_states, n_states, 3))
    for i in range(iter_max):
        obs = env.reset()
        total_reward = 0
        ## eta: learning rate is decreased at each step
        eta = max(min_lr, initial_lr * (0.85 ** (i//100)))
        for j in range(t_max):
            a, b = obs_to_state(env, obs)
            if np.random.uniform(0, 1) < eps:
                action = np.random.choice(env.action_space.n)
            else:
                logits = q_table[a][b]
                logits_exp = np.exp(logits)
                probs = logits_exp / np.sum(logits_exp)
                action = np.random.choice(env.action_space.n, p=probs)
            obs, reward, done, _ = env.step(action)
            total_reward += (gamma ** j) * reward
            # update q table
            a_, b_ = obs_to_state(env, obs)
            q_table[a][b][action] = q_table[a][b][action] + eta * (reward + gamma *  np.max(q_table[a_][b_]) - q_table[a][b][action])
            if done:
                break
        if i % 100 == 0:
            print('Iteration #%d -- Total reward = %d.' %(i+1, total_reward))
    solution_policy = np.argmax(q_table, axis=2)
    solution_policy_scores = [run_episode(env, solution_policy, False) for _ in range(100)]
    print("Average score of solution = ", np.mean(solution_policy_scores))
    # Animate it
    run_episode(env, solution_policy, True)

In [11]:
main_qlearn()

----- using Q Learning -----
Iteration #1 -- Total reward = -200.
Iteration #101 -- Total reward = -200.
Iteration #201 -- Total reward = -200.
Iteration #301 -- Total reward = -200.
Iteration #401 -- Total reward = -200.
Iteration #501 -- Total reward = -200.
Iteration #601 -- Total reward = -200.
Iteration #701 -- Total reward = -200.
Iteration #801 -- Total reward = -200.
Iteration #901 -- Total reward = -200.
Iteration #1001 -- Total reward = -200.
Iteration #1101 -- Total reward = -200.
Iteration #1201 -- Total reward = -200.
Iteration #1301 -- Total reward = -200.
Iteration #1401 -- Total reward = -200.
Iteration #1501 -- Total reward = -200.
Iteration #1601 -- Total reward = -178.
Iteration #1701 -- Total reward = -200.
Iteration #1801 -- Total reward = -200.
Iteration #1901 -- Total reward = -200.
Average score of solution =  -195.94
